## Import all the required packages and library 

In [1]:
import numpy as np
import os
import cv2
import glob
import itertools
import glob
import random
import matplotlib.pyplot as plt
from keras.models import *
from keras.layers import *
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_yaml

/home/bokoo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Pre-process the surgical tool images to fit to the Network 

In [2]:
def getImage(ImagePath, Width, Height):
    img = cv2.imread(ImagePath, 1)
    img = cv2.resize(img, ( Width , Height ))
    img = img.astype(np.float32)
    img[:,:,0] -= 103.939
    img[:,:,1] -= 116.779
    img[:,:,2] -= 123.68
    img = np.rollaxis(img, 2, 0)
    return img

# def getImage(ImagePath, Width, Height):
#     img = cv2.imread(ImagePath, 1)
#     img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
#     h,s,v=cv2.split(img_hsv)
#     ret,th = cv2.threshold(s,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
#     img=cv2.merge((img_gray,th,v))
#     img = np.float32(cv2.resize(img, ( Width , Height ))) / 127.5 - 1
#     img = np.rollaxis(img, 2, 0)
#     return img

## Get the segmentation Mask

In [3]:
def getSegmentationMask(SegImagePath, numberClass,  Width, Height):
    seg_labels = np.zeros((Height, Width, numberClass))
    img = cv2.imread(SegImagePath, 1)
    img = cv2.resize(img, ( Width, Height))
    img = img[:, :, 0]
    img = img/img.max()

    for class_label in range(numberClass):
        seg_labels[:, :, class_label] = (img == class_label ).astype(int)
    seg_labels = np.reshape(seg_labels, ( Width*Height , numberClass))
    return seg_labels

## Image Generator for the fit_generator

In [4]:

def Generator(ImagesPath, SegImagePath, batch_size, numberClass, Height, Width):
    assert ImagesPath[-1] == '/' # Image Path should be ended with /.
    assert SegImagePath[-1] == '/' # Image Path should be ended with /.
    
    images = glob.glob(ImagesPath+"*.jpg")+glob.glob(ImagesPath+"*.png")+glob.glob(ImagesPath+"*.jpeg")
    images.sort() # Sort the images 
    
    mask  = glob.glob(SegImagePath+"*.jpg")+glob.glob(SegImagePath+"*.png")+glob.glob(SegImagePath+"*.jpeg")
    mask.sort() # Sort the images
    
    # Total Number of images should be same as the total number of segmented Mask
    assert len(images) == len(mask) 

    
    # Checking that every image has their corresponding segmented mask.
    for im , seg in zip(images,mask):
        assert(  im.split('/')[-1].split(".")[0] ==  seg.split('/')[-1].split(".")[0] )
    
    zipped = itertools.cycle( zip(images,mask) )
        
    while True:
        ImageArray = []
        TargetArray = []
        for _ in range( batch_size) :
            im , seg = next(zipped)
            ImageArray.append(getImage(im, Width, Height))
            TargetArray.append(getSegmentationMask(seg, numberClass, 224, 224))
            yield np.array(ImageArray), np.array(TargetArray)

## Load the Train and Test image Directory

In [5]:
CurrentDirectory=os.getcwd()
TrainImagePath=CurrentDirectory+'/FINAL/TrainImage/'
TrainSegImagePath=CurrentDirectory+'/FINAL/TrainGT/'
TestImagePath=CurrentDirectory+'/FINAL/TestImage/'
TestSegImagePath=CurrentDirectory+'/FINAL/TestGT/'

## Create Model

In [6]:
VGG_Weights_path = CurrentDirectory+"/vgg16_weights_th_dim_ordering_th_kernels.h5"
# https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_th_dim_ordering_th_kernels.h5

IMAGE_ORDERING = 'channels_first'


def VGGUnet(n_classes, input_height=224, input_width=224 ,vgg_level=3):
    
    assert input_height%32 == 0
    assert input_width%32 == 0
    
    img_input = Input(shape=(3,input_height,input_width))
    
    # Block_1 of the VGG16
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1', data_format=IMAGE_ORDERING )(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2', data_format=IMAGE_ORDERING )(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool', data_format=IMAGE_ORDERING )(x)
    f1 = x
    
    # Block_2 of the VGG16
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1', data_format=IMAGE_ORDERING )(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2', data_format=IMAGE_ORDERING )(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool', data_format=IMAGE_ORDERING )(x)
    f2 = x
    
    # Block_3 of the VGG16
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1', data_format=IMAGE_ORDERING )(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2', data_format=IMAGE_ORDERING )(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3', data_format=IMAGE_ORDERING )(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool', data_format=IMAGE_ORDERING )(x)
    f3 = x
    
    # Block_4 of the VGG16
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1', data_format=IMAGE_ORDERING )(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2', data_format=IMAGE_ORDERING )(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3', data_format=IMAGE_ORDERING )(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool', data_format=IMAGE_ORDERING )(x)
    f4 = x
    
    
    # Block_5 of the VGG16
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1', data_format=IMAGE_ORDERING )(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2', data_format=IMAGE_ORDERING )(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3', data_format=IMAGE_ORDERING )(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool', data_format=IMAGE_ORDERING )(x)
    f5 = x
    
    
    #Fully Connected Layer of the VGG16\
    x = Flatten(name='flatten')(x)
    x = Dense(4096, activation='relu', name='fc1')(x)
    x = Dense(4096, activation='relu', name='fc2')(x)
    x = Dense( 1000 , activation='softmax', name='predictions')(x)
    
    #Create VGG16 Model to fit the weight
    vgg  = Model( img_input, x)
    vgg.load_weights(VGG_Weights_path)
#     levels = [f1 , f2 , f3 , f4 , f5 ]
    
    o = f4
#     o = (UpSampling2D( (2,2), data_format=IMAGE_ORDERING))(o)
    o = ( ZeroPadding2D( (1,1) , data_format=IMAGE_ORDERING ))(o)
    o = ( Conv2D(512, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
    o = ( BatchNormalization())(o)
    
    o = (UpSampling2D( (2,2), data_format=IMAGE_ORDERING))(o)
    o = ( concatenate([ o ,f3],axis=1 )  )
    o = ( ZeroPadding2D( (1,1), data_format=IMAGE_ORDERING))(o)
    o = ( Conv2D( 256, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
    o = ( BatchNormalization())(o)
    
    o = (UpSampling2D( (2,2), data_format=IMAGE_ORDERING))(o)
    o = ( concatenate([o,f2],axis=1 ) )
    o = ( ZeroPadding2D((1,1) , data_format=IMAGE_ORDERING ))(o)
    o = ( Conv2D( 128 , (3, 3), padding='valid' , data_format=IMAGE_ORDERING ) )(o)
    o = ( BatchNormalization())(o)
    
    o = (UpSampling2D( (2,2), data_format=IMAGE_ORDERING))(o)
    o = ( concatenate([o,f1],axis=1 ) )
    o = ( ZeroPadding2D((1,1)  , data_format=IMAGE_ORDERING ))(o)
    o = ( Conv2D( 64 , (3, 3), padding='valid'  , data_format=IMAGE_ORDERING ))(o)
    o = ( BatchNormalization())(o)
    
    o = (UpSampling2D( (2,2), data_format=IMAGE_ORDERING))(o)
    o = ( ZeroPadding2D((1,1)  , data_format=IMAGE_ORDERING ))(o)
    o = ( Conv2D( 64 , (3, 3), padding='valid'  , data_format=IMAGE_ORDERING ))(o)
    o = ( BatchNormalization())(o)
    
    o =  Conv2D( n_classes , (3, 3) , padding='same', data_format=IMAGE_ORDERING )( o )
    o_shape = Model(img_input , o ).output_shape
    
    outputHeight = o_shape[2]
    outputWidth = o_shape[3]
    o = (Reshape((  n_classes , outputHeight*outputWidth   )))(o)
    
    o = (Permute((2, 1)))(o)
    o = (Activation('softmax'))(o)
    model = Model( img_input , o )
    model.outputWidth = outputWidth
    model.outputHeight = outputHeight
    
    return model

## Set all the Hyper-paramters

In [7]:
numberClass=2
Height=224
Width=224
optimizer='adadelta'
loss='categorical_crossentropy'
metric=['acc']
train_batch_size=16
test_batch_size=11
epochs=100

## Load the Imges and create Generator

In [8]:
TrainGen=Generator(TrainImagePath, TrainSegImagePath,  train_batch_size,  numberClass, Height, Width)
numberTrainImages=len(glob.glob(TrainImagePath+"*.jpg")+glob.glob(TrainImagePath+"*.png")+glob.glob(TrainImagePath+"*.jpeg"))

TestGen=Generator(TestImagePath, TestSegImagePath,  test_batch_size,  numberClass, Height, Width)
numberTestImages=len(glob.glob(TestImagePath+"*.jpg")+glob.glob(TestImagePath+"*.png")+glob.glob(TestImagePath+"*.jpeg"))

## Saving and Early stopping of the model

In [9]:
# I stopped training automagically with EarlyStopping after 10 consecutive epochs without improvement
early_stopping = EarlyStopping(monitor='val_acc', patience=10, verbose=1, mode='max',restore_best_weights=True)


modelName='SavedModel/VGGUnetModel'
SaveweightName= modelName+'.hdf5'
SaveModelName=modelName+'.yaml'

SavingbestModel = ModelCheckpoint(SaveweightName, monitor='val_acc', verbose=1, save_best_only=True, mode='max')


## Customize loss function and optimizer

In [10]:
def dice_coef(y_true, y_pred,smooth=1):
    y_true_f = K.flatten(y_true) 
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection+smooth) / (K.sum(y_true_f) + K.sum(y_pred_f)+smooth) 

def dice_coef_loss(y_true, y_pred): 
    return 1-dice_coef(y_true, y_pred) 

# model.compile(optimizer=optimizer, loss=dice_coef_loss, metrics=[dice_coef])

## Compile the Model and Training

In [11]:
model = VGGUnet(numberClass, input_height=Height, input_width=224)

model.compile(loss=loss, optimizer= optimizer, metrics=metric)
model.summary()
model_yaml = model.to_yaml()
with open(SaveModelName, "w") as yaml_file:
    yaml_file.write(model_yaml)

# history=model.fit_generator(TrainGen,
#                          steps_per_epoch= 10*(numberTrainImages//train_batch_size),
#                          epochs=epochs,
#                          verbose=1,
#                          validation_data=TestGen,
#                          validation_steps=numberTestImages//test_batch_size,
#                          callbacks=[early_stopping,SavingbestModel]
#                         )


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 3, 224, 224)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 64, 224, 224) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 64, 224, 224) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 64, 112, 112) 0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

## Plotting Traing and Validation History

In [12]:
# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.grid('on')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.grid('on')
plt.show()

NameError: name 'history' is not defined

In [ ]:
model.summary()